# Extract torso part from PCA model and save in .mat format
- meanshape coordinates
- evalues
- evectors

In [4]:
def is_close_relative(value_a, value_b, tolerance=1e-4):
    """Check if two values are close to each other within a certain relative tolerance."""
    if value_a == value_b == 0:
        return True
    relative_error = abs(value_a - value_b) / max(abs(value_a), abs(value_b))
    return relative_error < tolerance

def filter_data_relative_precision(mean_shape, eigenvectors, csv_points, tolerance=1e-4):
    # Extracting points from the DataFrame
    points_from_csv = csv_points[['Points_0', 'Points_1', 'Points_2']].values
    
    # Number of points in the mean_shape
    num_points = len(mean_shape) // 3
    
    # Indexes to be kept
    indexes_to_keep = []
    
    # Identifying points to keep
    for i in range(num_points):
        point = mean_shape[i], mean_shape[i + num_points], mean_shape[i + 2*num_points]
        if any(all(is_close_relative(point[dim], p[dim], tolerance) for dim in range(3)) for p in points_from_csv):
            indexes_to_keep.append(i)
    
    # Filtering mean_shape
    filtered_mean_shape = np.concatenate([mean_shape[indexes_to_keep], 
                                          mean_shape[num_points:2*num_points][indexes_to_keep], 
                                          mean_shape[2*num_points:][indexes_to_keep]])
    
    # Filtering eigenvectors
    filtered_eigenvectors = np.concatenate([eigenvectors[:, indexes_to_keep], 
                                            eigenvectors[:, num_points:2*num_points][:, indexes_to_keep], 
                                            eigenvectors[:, 2*num_points:][:, indexes_to_keep]], axis=1)
    
    return filtered_mean_shape, filtered_eigenvectors

In [5]:
import pandas as pd
import numpy as np
# from PCA_model import PCA_model, registration_to_strocchi, save_points_to_vtk


# caeser data from Internet: 
# https://humanshape.mpi-inf.mpg.de/
# path_to_meanshape_file = r'C:\Users\xuhu\pyheart-lib\examples\preprocessor\PCA_data\meanShape.mat'
path_to_meanshape_file = r'D:\Data\SSM_Input\input\meanShape.mat'
path_to_evectors = r'D:\Data\SSM_Input\input\evectors.mat'
path_to_evalues = r'D:\Data\SSM_Input\input\evalues.mat'


# file use for extract torso part from meanshape
torso_csv_file_path = r"D:\Data\SSM_Input\input\Torso.csv"
torso_mat_file_path = 'New_torso.mat'
torso_evectors_mat_file_path = 'Torso_evectors.mat'

# transfer_matrix_file_path = r'C:\Users\xuhu\pyheart-lib\examples\preprocessor\PCA_data\combined_transform_matrix.csv'


In [6]:
from scipy.io import loadmat, savemat

meanshape_model = loadmat(path_to_meanshape_file)['points']
meanshape_evectors = loadmat(path_to_evectors)['evectors']
meanshape_evalues = loadmat(path_to_evalues)['evalues']

torso_model = meanshape_model.copy()
test_torso_model = np.hstack(torso_model.T.flatten())
csv_points_df = pd.read_csv(torso_csv_file_path)
torso, torso_evectors = filter_data_relative_precision(test_torso_model, meanshape_evectors, csv_points_df)

savemat(torso_mat_file_path, {'points': csv_points_df[['Points_0', 'Points_1', 'Points_2']].values})
savemat(torso_evectors_mat_file_path, {'evectors': torso_evectors})